In [1]:
import sys
import os

thesis_path = "/" + os.path.join(
    *os.getcwd().split(os.path.sep)[:-1])
sys.path.append(thesis_path)


In [7]:
import pandas as pd
import pickle 
from glob import glob
from models.metrics import ASPMetrics

all_metrics = []

fp = ['', 'Users', 'robinloebbert', 'Masterarbeit', 'thesis', 'experiments', '02_content', 'data', 'seed_2', '04_metrics', 'size_8000', 'error_ratio_0', 'best_sampled_search_test.pkl']

for metrics_filepath in glob(os.path.join(thesis_path, "experiments", "02_content", "data", r"**", "04_metrics", "**", r"*.pkl"), recursive=True):
    fp = metrics_filepath.split(os.path.sep)
    with open(metrics_filepath, "rb") as file:
        metrics: ASPMetrics = pickle.load(file)
    f1 = float(metrics.f1().cpu().numpy())
    name = fp[-1].split("_")
    timestep = 0
    if name[1] == "sampled":
        timestep = 1
    elif name[1] == "full":
        timestep = 2
    all_metrics.append({
        "seed": int(fp[-5][-1]),
        "checkpoint": name[0],
        "gazetteer_size": int(fp[-3].split("_")[1]),
        "error_ratio": int(fp[-2].split("_")[2]),
        "timestep": timestep,
        "dataset": fp[-1][5:].split(".")[0],
        "f1": f1
    })
all_metrics

[{'seed': 2,
  'checkpoint': 'best',
  'gazetteer_size': 8000,
  'error_ratio': 0,
  'timestep': 1,
  'dataset': 'sampled_search_test',
  'f1': 0.8623999953269958},
 {'seed': 2,
  'checkpoint': 'best',
  'gazetteer_size': 8000,
  'error_ratio': 0,
  'timestep': 0,
  'dataset': 'error_search_error_dev',
  'f1': 0.8622999787330627},
 {'seed': 2,
  'checkpoint': 'last',
  'gazetteer_size': 8000,
  'error_ratio': 0,
  'timestep': 2,
  'dataset': 'full_search_dev',
  'f1': 0.8748999834060669},
 {'seed': 2,
  'checkpoint': 'last',
  'gazetteer_size': 8000,
  'error_ratio': 0,
  'timestep': 1,
  'dataset': 'sampled_search_dev',
  'f1': 0.8629999756813049},
 {'seed': 2,
  'checkpoint': 'last',
  'gazetteer_size': 8000,
  'error_ratio': 0,
  'timestep': 0,
  'dataset': 'error_search_test',
  'f1': 0.8622999787330627},
 {'seed': 2,
  'checkpoint': 'best',
  'gazetteer_size': 8000,
  'error_ratio': 0,
  'timestep': 1,
  'dataset': 'sampled_search_train',
  'f1': 0.9702000021934509},
 {'seed': 2,


In [10]:
metrics_df = pd.DataFrame.from_records(all_metrics)
metrics_df.head()

,seed,checkpoint,gazetteer_size,error_ratio,timestep,dataset,f1
0,2,best,8000,0,1,sampled_search_test,0.8624
1,2,best,8000,0,0,error_search_error_dev,0.8623
2,2,last,8000,0,2,full_search_dev,0.8749
3,2,last,8000,0,1,sampled_search_dev,0.8630
4,2,last,8000,0,0,error_search_test,0.8623


In [11]:
metrics_df.describe()

,seed,gazetteer_size,error_ratio,timestep,f1
count,864.00000,864.000000,864.000000,864.00000,864.000000
mean,2.00000,5000.000000,7.500000,1.00000,0.879439
std,0.81697,2237.363123,5.593408,0.81697,0.050166
min,1.00000,2000.000000,0.000000,0.00000,0.717500
25%,1.00000,3500.000000,3.750000,0.00000,0.851300
50%,2.00000,5000.000000,7.500000,1.00000,0.864450
75%,3.00000,6500.000000,11.250000,2.00000,0.924850
max,3.00000,8000.000000,15.000000,2.00000,0.973500


In [15]:
metrics_df.iloc[metrics_df[(metrics_df["timestep"]==2) & (metrics_df["dataset"].str.endswith("test"))]["f1"].idxmin()]

seed                             2
checkpoint                    last
gazetteer_size                4000
error_ratio                     15
timestep                         2
dataset           full_search_test
f1                           0.818
Name: 276, dtype: object